### Importing libraries

In [1]:
import os, sys, time, datetime, glob, bcolz, random

In [2]:
import keras
import numpy as np
import pandas

Using TensorFlow backend.


In [3]:
from keras import Sequential

In [4]:
import matplotlib.pyplot as plt

In [12]:
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Convolution2D

In [20]:
folder_train_data = '/media/amogh/Stuff/CMU/datasets/AffectNet/Manually_Annotated_Images/'

### Loading data

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    shear_range=0.2,/media/amogh/Stuff/CMU/datasets/AffectNet/Manually_Annotated_Images
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [22]:
def get_batches(dirname, gen=ImageDataGenerator(), shuffle=True, batch_size=4, class_mode='categorical', target_size=(224,224)):
    return gen.flow_from_directory(dirname, target_size=target_size, class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [26]:
def get_data(path, target_size=(224,224)):
    batches = get_batches(path, shuffle=False, batch_size=1, class_mode=None, target_size=target_size)
    return np.concatenate([batches.next() for i in range(batches.samples)])

In [27]:
trn = get_data(folder_train_data)

Found 38332 images belonging to 74 classes.


KeyboardInterrupt: 

In [ ]:
type(trn)

In [29]:
import gc
gc.collect()

0

### Loading a pretrained model

Loading VGG16 model

In [8]:
model = applications.VGG16(weights='imagenet', include_top=False)
print ('Model loaded')

58892288/58889256 [==============================] - 22s 0us/step
Model loaded


Splitting the layers, and making a Conv model

In [13]:
layers = model.layers
layer_idx = [index for index,layer in enumerate(layers) if type(layer) is Convolution2D][-1]
conv_layers, fc_layers = layers[:layer_idx+1], layers[layer_idx+1:]

In [15]:
conv_model = Sequential(conv_layers)